In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/8.url'):
    shutil.rmtree('./file/8.url')

os.makedirs('./file/8.url')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./file/7.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =


work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            list_url = []
            while len(list_url) < input_.loc[a, 'SKU Count']:
                for i in range(input_.loc[a, 'Page']):
                    b = 0
                    while True:
                        b += 1

                        try:
                            resp = requests.get(f'''{input_.loc[a, 'Url']}&start={i*100}&num=100''',
                                                 headers=get_header(),
                                                 proxies=get_proxy(),
                                                 timeout=(10, 10))
                            
                            if resp.status_code == 200:
                                soup = BeautifulSoup(resp.text, 'lxml')
                                html = etree.HTML(str(soup))

                                list_href = html.xpath('//h2[@class="item-headline"]/a/@href')
                                if list_href:
                                    break
                        except KeyboardInterrupt:
                            break
                        except:
                            continue
            
                    # = = = = = = = = = = = = = = =

                    list_url += [f'https://www.dormanproducts.com/{href.strip()}' for href in list_href]
                    list_url = sorted(list(set(list_url)), key=list_url.index)
            
                    # = = = = = = = = = = = = = = =

                    if work.qsize() % 100 == 0 and work.qsize() != 0:
                        clear_output()

                    print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']} > Page {i+1}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                    # = = = = = = = = = = = = = = =
            
                    if len(list_url) == input_.loc[a, 'SKU Count']:
                        break
                        
            # = = = = = = = = = = = = = = =

            output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                                   'Url': list_url})

            output.to_excel(f'''./file/8.url/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-url_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/8.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/8.url/{file}',
              f'''./file/8.url/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[尝试次数：1] - 2174.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Vacuum Reservoir Nut > Page 1
[剩余数量：100] - [当前时间：00:03:53]

[尝试次数：10] - 1919.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Suspension - Stabilizer Bar Link Bushing Kit > Page 7
[剩余数量：99] - [当前时间：00:03:54]

[尝试次数：9] - 2172.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Vacuum Reservoir > Page 1
[剩余数量：99] - [当前时间：00:03:54]

[尝试次数：28] - 1779.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Steering - Tie Rod End > Page 58
[剩余数量：98] - [当前时间：00:03:55]

[尝试次数：14] - 1920.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Suspension - Stabilizer Bar Link Kit > Page 39
[剩余数量：98] - [当前时间：00:03:55]

[尝试次数：1] - 2149.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Turn Signal Light > Page 5
[剩余数量：98] - [当前时间：00:03:55]

[尝试次数：1] - 1912.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Suspension - Stabilizer Bar > Page 21
[剩余数量：98] - [当前时间：00:03:56]

[尝试次数：1] - 17

Progress: 100%|████████████████████████| 2279/2279 [00:00<00:00, 8276.53it/s]

Done ~
